<a href="https://colab.research.google.com/github/niikun/langchain_tutorial/blob/main/Build_an_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build an Agent
出展:[https://python.langchain.com/v0.2/docs/tutorials/agents/]  

言語モデルはそれ自体ではアクションを起こすことができない。LangChainの大きなユースケースはエージェントの作成です。エージェントは、LLMを推論エンジンとして使用し、どのアクションを取るか、どの入力を渡すかを決定するシステムです。アクションを実行した後、その結果をLLMにフィードバックし、さらにアクションが必要か、あるいは終了しても問題ないかを判断することができます。

このチュートリアルでは、検索エンジンと対話できるエージェントを作ります。このエージェントに質問をしたり、検索ツールを呼び出すのを見たり、会話をしたりすることができます。  

## Concepts  
このチュートリアルでは、以下の方法を学びます：

- 言語モデル、特にツール呼び出し機能を使用する
- 検索ツールを使ってインターネットから情報を検索する。
- LangGraphエージェントを構成し、LLMを使ってアクションを決定し、実行する。
- LangSmithを使ったアプリケーションのデバッグとトレース

# End-to-end agent
以下のコード・スニペットは、どのツールを使用するかを決定するためにLLMを使用する、完全に機能するエージェントを表しています。一般的な検索ツールを備えている。会話記憶を持っており、マルチターンチャットボットとして使うことができます。

このガイドの残りの部分では、個々のコンポーネントと各パーツが何をするかについて説明します！

In [ ]:
# Import relevant functionality
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import chat_agent_executor

# Create the agent
memory = SqliteSaver.from_conn_string(":memory:")
model = ChatAnthropic(model_name="claude-3-sonnet-20240229")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = chat_agent_executor.create_tool_calling_executor(
    model, tools, checkpointer=memory
)

# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
):
    print(chunk)
    print("----")

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]}, config
):
    print(chunk)
    print("----")

ModuleNotFoundError: No module named 'langchain_anthropic'

LangGraphは、複数の言語モデル（LLM）を使った状態を持つマルチエージェントアプリケーションを構築するためのライブラリです。これはLangChainの拡張として機能し、非線形なフローを持つ複雑な状態管理が必要なアプリケーションに適しています。

具体的には、以下のような用途があります：

- マルチエージェントワークフロー: 複数のエージェントが協調してタスクを分割して解決するシステムを構築するのに適しています。各エージェントは共有された状態と異なるツールを使用して協力します​ (GitHub)​​ (Analytics Vidhya)​。

- 状態管理と中断の処理: LangGraphは、エージェントが対話やツールの使用を通じて複数のターンにわたって状態をシームレスに管理するための機能を提供します。これにより、プロセスを一時停止したり、人間の介入を挟んだりすることが容易になります​ (LangChain AI)​​ (GitHub)​。

- サイクルと持続性: LangGraphは、繰り返しのループや持続性を必要とするアプリケーションに対して特に有効です。これにより、長期間にわたるマルチセッションアプリケーションの構築が容易になります​ (GitHub)​​ (Analytics Vidhya)​。

- ツールの柔軟な利用: LangGraphは、ツールの呼び出しや動的なルーティングを柔軟に管理するためのノードとエッジを定義することで、LLMを使ったエージェントの動作を細かく制御することができます​ (LangChain AI)​​ (GitHub)​。

例えば、LangGraphを使ってカスタマーサポートエージェントを構築する場合、エージェントは質問に答えるためにツールを使用したり、必要に応じて人間のオペレーターと連携したりできます。プロセスを一時停止して、後で再開することも簡単です。

LangGraphを使用することで、複雑な状態管理やエージェントの協調動作が求められるアプリケーションの開発がより効率的かつ効果的になります。詳細なドキュメントやチュートリアルについては、LangGraphの公式ドキュメントを参照してください​ (LangChain AI)​​ (GitHub)​。

## Instllation  
LangChain

In [ ]:
!pip install -U langchain-community langgraph langchain-anthropic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 862.7/862.7 kB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
from google.colab import userdata
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

## Tavily Search  
AIエージェント（LLM）のために特別に構築された検索エンジンでAIの能力を高め、リアルタイムで正確かつ事実に基づいた結果を迅速に提供します。Search APIは、LLMとAIアプリケーションを信頼できるリアルタイムの知識に接続し、幻覚や全体的なバイアスを減らすのに役立ちます。簡単に言えば、このAPIはAIがより良い意思決定をするのを助けます.  
使い方：  
https://python.langchain.com/v0.2/docs/integrations/tools/tavily_search/  
API:  
https://tavily.com/

In [ ]:
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

## Define tools  
まず、使いたいツールを作る必要がある。主なツールは検索エンジンのTavilyです。LangChainにはTavily検索エンジンをツールとして簡単に使うための組み込みツールがあります。

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [ ]:
search = TavilySearchResults(max_results=2)

In [ ]:
search.invoke("what is the weather in Tokyo")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'Tokyo', 'region': 'Tokyo', 'country': 'Japan', 'lat': 35.69, 'lon': 139.69, 'tz_id': 'Asia/Tokyo', 'localtime_epoch': 1717851864, 'localtime': '2024-06-08 22:04'}, 'current': {'last_updated_epoch': 1717851600, 'last_updated': '2024-06-08 22:00', 'temp_c': 22.4, 'temp_f': 72.3, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 11.9, 'wind_kph': 19.1, 'wind_degree': 180, 'wind_dir': 'S', 'pressure_mb': 1016.0, 'pressure_in': 30.0, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 78, 'cloud': 50, 'feelslike_c': 24.7, 'feelslike_f': 76.5, 'windchill_c': 21.5, 'windchill_f': 70.7, 'heatindex_c': 24.5, 'heatindex_f': 76.0, 'dewpoint_c': 15.6, 'dewpoint_f': 60.2, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 1.0, 'gust_mph': 18.1, 'gust_kph': 29.1}}"},
 {'url': 'https://www.accuweather.com/en/jp/tokyo/226396/august-weather/226396',
  'c

必要であれば、他のツールを作ることもできる。必要なツールがすべて揃ったら、後で参照できるようにリストに入れることができる。

In [ ]:
tools = [search]

## Using Language Models  
次に、ツールを呼び出すための言語モデルの使い方を学びましょう。LangChainは様々な言語モデルをサポートしています！

In [ ]:
!pip install -qU langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.4 MB/s eta 0:00:00


In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

In [ ]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="こんにちは！")])
response.content

'こんにちは！お元気ですか？何かお手伝いできることがあれば教えてくださいね。'

このモデルでツールを呼び出せるようにするにはどうすればいいか、見てみよう。それを可能にするために、.bind_toolsを使い、言語モデルにこれらのツールに関する知識を与えている。

In [ ]:
model_with_tools = model.bind_tools(tools)

これでモデルを呼び出すことができる。まずは通常のメッセージで呼び出し、どのように応答するかを見てみましょう。  
`content`フィールドと`tool_calls`フィールドの両方を見ることができます。

In [ ]:
response = model_with_tools.invoke([HumanMessage(content="こんにちは！")])
print(f"Content:{response.content}")
print(f"Tool calls:{response.tool_calls}")

Content:こんにちは！今日はどのようなお手伝いができるでしょうか？
Tool calls:[]


では、ツールが呼び出されることを期待するような入力で呼び出してみよう。

In [ ]:
response = model_with_tools.invoke([HumanMessage(content="東京の天気を教えて？")])
print(f"Content:{response.content}")
print(f"Tool calls:{response.tool_calls}")

Content:
Tool calls:[{'name': 'tavily_search_results_json', 'args': {'query': 'Tokyo weather'}, 'id': 'call_iNm5avFVcHqfLV9fCTQ6Ewh3'}]


コンテンツはありませんが、ツールの呼び出しはあります！それはTavily Searchツールを呼び出すことを望んでいる。

これはまだツールを呼び出していません。実際に呼び出すには、エージェントを作成します。

## Create the Agent
ツールとLLMを定義したので、エージェントを作成します。エージェントの構築にはLangGraphを使います。現在、エージェントを構築するために高レベルのインタフェースを使用していますが、LangGraphの良いところは、この高レベルのインタフェースが、エージェントのロジックを変更したい場合のために、低レベルで高度に制御可能なAPIに支えられていることです。  
`model_with_tools`ではなく、`model`を渡していることに注意してください。それは、`create_tool_calling_executor`が私たちのために`bind_tools`を呼び出すからです。


In [ ]:
from langgraph.prebuilt import create_react_agent
agent_executor = create_react_agent(model,tools)

## Run the Agent
これでエージェントをいくつかのクエリで実行できるようになりました！今のところ、これらはすべてステートレスクエリであることに注意してください（以前のインタラクションを覚えていません）。エージェントはインタラクションの最後に最終的な状態を返すことに注意してください（これはあらゆる入力を含みますが、出力だけを取得する方法は後で説明します）。

まず最初に、ツールを呼び出す必要がないときにどのように応答するかを見てみましょう：

In [ ]:
response = agent_executor.invoke({"messages":[HumanMessage(content="こんにちは‼")]})
response['messages']

[HumanMessage(content='こんにちは‼', id='cc6328ac-6f0b-4a29-b0fe-cc4044b148ce'),
 AIMessage(content='こんにちは！今日はどんなお手伝いができますか？', response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 81, 'total_tokens': 95}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-56cc7b01-6ddf-4c2e-85dc-10283311e86e-0', usage_metadata={'input_tokens': 81, 'output_tokens': 14, 'total_tokens': 95})]

In [ ]:
response = agent_executor.invoke(
    {"messages":[HumanMessage(content="東京の天気を教えて")]}
)
response['messages']

[HumanMessage(content='東京の天気を教えて', id='eebbef85-e270-47e9-b34a-f88ce5012d30'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_LWfVlKa8zBvadN163MZTPnVM', 'function': {'arguments': '{"query":"東京 天気"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 86, 'total_tokens': 106}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-70c56914-eb72-4833-8e1b-99606fdd1dbd-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '東京 天気'}, 'id': 'call_LWfVlKa8zBvadN163MZTPnVM'}], usage_metadata={'input_tokens': 86, 'output_tokens': 20, 'total_tokens': 106}),
 ToolMessage(content='[{"url": "https://weathernews.jp/onebox/35.691667/139.750000/q=\\u6771\\u4eac&v=7568ec9017b1ee619b719b2e27bb2222ff3533c3d047419f40d2980258c0a799&temp=c&lang=ja", "content": "\\u5348\\u5f8c20%\\n\\u65e5\\u5dee\\u3057\\u306b\\u669

## Streaming Messages
エージェントを`invoke`で呼び出して、最終的なレスポンスを返す方法を見ました。エージェントが複数のステップを実行する場合、時間がかかるかもしれません。途中経過を表示するために、発生したメッセージをストリームバックすることができます。

In [ ]:
for chunk in agent_executor.stream(
    {"messages":[HumanMessage(content="Tell me the weather in Tokyo tomorrow in Japanese.")]}
):
    print(chunk)
    print("========")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_w4qTdFC2P7IT0C0UYdXF77mD', 'function': {'arguments': '{"query":"Tokyo weather forecast for tomorrow"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 89, 'total_tokens': 111}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f17dded4-4869-4825-82f8-f8628f09c996-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Tokyo weather forecast for tomorrow'}, 'id': 'call_w4qTdFC2P7IT0C0UYdXF77mD'}], usage_metadata={'input_tokens': 89, 'output_tokens': 22, 'total_tokens': 111})]}}
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Tokyo\', \'region\': \'Tokyo\', \'country\': \'Japan\', \'lat\': 35.69, \'lon\': 139.69, \'tz_id\': \'Asia/Tokyo\', \'l

## Streaming tokens
メッセージをストリームバックするだけでなく、トークンをストリームバックするのも便利です。`.astream_events`メソッドでこれを行うことができる。




In [ ]:
async for event in agent_executor.astream_events(
    {"messages":[HumanMessage(content="What's the weather in tokyo ,tell me in Japanese.")]},version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"]=="Agent"
        ):
            print(
                f'Starting Agent : {event["name"]} with input : {event["data"].get("input")}'
            )
    elif kind == "on_chain_end":
        if (
            event["name"]=="Agent"
        ):
            print()
            print("------------")
            print(
                f'Done Agent : {event["name"]} with output : {event["data"].get("output")["output"]}'
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


--
Starting tool: tavily_search_results_json with inputs: {'query': 'Tokyo weather'}
Done tool: tavily_search_results_json
Tool output was: [{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Tokyo', 'region': 'Tokyo', 'country': 'Japan', 'lat': 35.69, 'lon': 139.69, 'tz_id': 'Asia/Tokyo', 'localtime_epoch': 1717854420, 'localtime': '2024-06-08 22:47'}, 'current': {'last_updated_epoch': 1717854300, 'last_updated': '2024-06-08 22:45', 'temp_c': 22.4, 'temp_f': 72.3, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 10.5, 'wind_kph': 16.9, 'wind_degree': 160, 'wind_dir': 'SSE', 'pressure_mb': 1016.0, 'pressure_in': 30.0, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 78, 'cloud': 25, 'feelslike_c': 24.7, 'feelslike_f': 76.5, 'windchill_c': 21.5, 'windchill_f': 70.7, 'heatindex_c': 24.5, 'heatindex_f': 76.0, 'dewpoint_c': 15.6, 'dewpoint_f': 60.2, 'vis_km': 10.0, 'vis_miles': 6

## Adding in memory
先に述べたように、このエージェントはステートレスである。つまり、以前のやりとりを覚えていない。このエージェントにメモリーを与えるには、チェックポインターを渡す必要がある。チェックポインター`thread_id`も渡す必要がある（どのスレッド／会話から再開するかを知るため）

In [30]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

In [32]:
agent_executor = create_react_agent(model,tools,checkpointer=memory)
config = {"configurable":{"thread_id":"abc123"}}

In [33]:
for chunk in agent_executor.stream(
    {"messages":[HumanMessage(content="hi im niikun!")]},config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello, Niikun! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 84, 'total_tokens': 97}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-af723528-9822-4db8-ae8e-a8794be4464d-0', usage_metadata={'input_tokens': 84, 'output_tokens': 13, 'total_tokens': 97})]}}
----


In [35]:
for chunk in agent_executor.stream(
    {"messages":[HumanMessage(content="Whats my name?")]},config
):
    print(chunk)
    print("------")

{'agent': {'messages': [AIMessage(content='You mentioned that your name is Niikun. How can I help you today, Niikun?', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 108, 'total_tokens': 128}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-73bfad7d-e846-48ca-b71b-65ef3b2ccc58-0', usage_metadata={'input_tokens': 108, 'output_tokens': 20, 'total_tokens': 128})]}}
------


新しい会話を始めたければ、使用する`thread_id`を変更するだけでいい。

In [40]:
config = {"configurable":{"thread_id":"xyz123"}}

for chunk in agent_executor.stream(
    {"messages":[HumanMessage(content="whats my name? Tell me in Japanese")]},config
):
    print(chunk)

{'agent': {'messages': [AIMessage(content='あなたの名前は何ですか？\n(Anata no namae wa nan desu ka?)\n\nThis means "What is your name?" in Japanese.', response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 212, 'total_tokens': 244}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'stop', 'logprobs': None}, id='run-f1a5ac5a-de65-4eb5-9466-e26cd5758cfd-0', usage_metadata={'input_tokens': 212, 'output_tokens': 32, 'total_tokens': 244})]}}


## Conclusion
以上です！このクイックスタートでは、シンプルなエージェントの作成方法を説明しました。そして、中間ステップだけでなく、トークンも含めてレスポンスをストリームバックする方法を紹介しました！さらに、エージェントと会話ができるように、メモリも追加しました。エージェントは複雑なトピックで、学ぶべきことがたくさんあります！

エージェントについての詳しい情報は、LangGraphのドキュメントをご覧ください。これには独自のコンセプト、チュートリアル、ハウツーガイドがあります。